<a href="https://colab.research.google.com/github/ancestor9/2025_Fall_AI-Model-Operations-MLOps/blob/main/week07/python_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

database.py [링크](https://github.com/ancestor9/2025_Fall_AI-Model-Operations-MLOps/blob/main/week07/src/database.py)에 있는 코드를 copy -> paste

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

# 데이터베이스 연결 URL
# mvc_app.db 파일을 만들고 SQLite DB 연결 📞 전화기 만들기 (DB와 통화할 도구)
SQLALCHEMY_DATABASE_URL = "sqlite:///./mvc_app.db"

# engine = DB와 소통할 "연결 통로" 완성 ✅ 전화기가 완성됨!
engine = create_engine(
    SQLALCHEMY_DATABASE_URL,
    connect_args={"check_same_thread": False} # SQLite 설정
)

# SessionLocal = "세션을 찍어내는 공장" 🏭 세션 공장 완성 (필요할 때마다 세션 발급)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# SQLAlchemy 모델의 기본 클래스
# Base = "모든 테이블의 기본 설계도"📐 기본 건축 설계도
Base = declarative_base()
# -----> models.py의 class DBUser(Base) = "users 테이블 설계도" 🏠 users 테이블 청사진

# FastAPI DB 세션 의존성 함수 (Controller에 주입됨)
# 📍 세션 제공 함수
def get_db():
    db = SessionLocal()     # ① 새 세션 발급
    try:
        yield db            # ② 세션 제공 (임시 사용권)
    finally:
        db.close()          # ③ 세션 반납 (자동 정리)

# 실행 순서,코드 라인,무슨 일이 일어나나?,쉬운 비유,타이밍
# ① 시작,db = SessionLocal(),공장에서 새 세션 발급,📦 새 도서관 대출권 발급,요청 받자마자
# ② 제공,yield db,"""이 세션 써요!""",📖 책 건네주기,엔드포인트 실행 중
# ③ 정리,db.close(),세션 자동 반납,📚 책 반납 (자동),응답 끝난 후
# 🎯 핵심: yield = "임시로 빌려주고, 끝나면 자동 정리!" FastAPI가 이 마법을 알아서 처리해줘요

/tmp/ipython-input-876353234.py:20: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
engine

Engine(sqlite:///./mvc_app.db)

In [14]:
SessionLocal

sessionmaker(class_='Session', autocommit=False, bind=Engine(sqlite:///./mvc_app_item_setup.db), autoflush=False, expire_on_commit=True)

1. DB에 접속하여 user라는 테이블을 만드는 명령어

In [4]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import sessionmaker, Session
from sqlalchemy.ext.declarative import declarative_base


# 📦 모델 정의 for User table
class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True, index=True)
    username = Column(String, unique=True, index=True)
    email = Column(String, unique=True, index=True)
    hashed_password = Column(String)

# 🚀 DB 초기화: 테이블 생성 (Using Base.metadata)
def init_db():
    # Create all tables defined in Base.metadata
    Base.metadata.create_all(bind = engine)

# Create the user table
init_db()

print("User table created successfully!")

User table created successfully!


2. user 테이블에 record 2개를 생성하는 코드

In [5]:
from sqlalchemy.orm import Session

# 🔑 세션 관리 함수 (Using the get_db function defined earlier)
def get_session_context() -> Session:
    """
    Opens a session, uses it, and ensures it's closed.
    """
    db = SessionLocal() # Create a session
    try:
        return db
    finally:
        db.close() # Close the session

# 🧪 실행 로직
print("--- Creating users ---")

# 1. Get a session context
db_session = get_session_context()

try:
    # Create two User objects
    user1 = User(username="user1", email="user1@example.com", hashed_password="password1")
    user2 = User(username="user2", email="user2@example.com", hashed_password="password2")

    # Add the users to the session
    db_session.add(user1)
    db_session.add(user2)

    # Commit the transaction
    db_session.commit()

    print("✅ Two users created successfully!")

except Exception as e:
    # Rollback in case of error
    db_session.rollback()
    print(f"❌ Error creating users: {e}")

finally:
    # Close the session
    db_session.close()

print("--- User creation completed ---")

--- Creating users ---
✅ Two users created successfully!
--- User creation completed ---


3. user 테이블을 모든 records를 조회하는 코드

In [6]:
# 🧪 실행 로직
print("--- Retrieving users ---")

# 1. Get a session context
db_session = get_session_context()

try:
    # Query all users
    users = db_session.query(User).all()

    if users:
        print("✅ Retrieved users:")
        for user in users:
            print(f"  ID: {user.id}, Username: {user.username}, Email: {user.email}")
    else:
        print("🔎 No users found in the table.")

except Exception as e:
    print(f"❌ Error retrieving users: {e}")

finally:
    # Close the session
    db_session.close()

print("--- User retrieval completed ---")

--- Retrieving users ---
✅ Retrieved users:
  ID: 1, Username: user1, Email: user1@example.com
  ID: 2, Username: user2, Email: user2@example.com
--- User retrieval completed ---


4. ID 1을 지우는 코드

In [7]:
# 🧪 실행 로직
print("--- Deleting user with ID 1 ---")

# 1. Get a session context
db_session = get_session_context()

try:
    # Find the user with ID 1
    user_to_delete = db_session.query(User).filter(User.id == 1).first()

    if user_to_delete:
        # Delete the user
        db_session.delete(user_to_delete)
        # Commit the transaction
        db_session.commit()
        print("✅ User with ID 1 deleted successfully!")
    else:
        print("🔎 User with ID 1 not found.")

except Exception as e:
    # Rollback in case of error
    db_session.rollback()
    print(f"❌ Error deleting user: {e}")

finally:
    # Close the session
    db_session.close()

print("--- User deletion completed ---")

--- Deleting user with ID 1 ---
✅ User with ID 1 deleted successfully!
--- User deletion completed ---


## **1. ORM 없이 코딩하기**
ORM(Object-Relational Mapping)은 객체 지향 프로그래밍 언어와 관계형 데이터베이스 사이의 호환되지 않는 데이터를 변환해주는 기술입니다. ORM을 사용하면 개발자는 SQL을 직접 작성하지 않고도 객체처럼 데이터베이스를 다룰 수 있어 생산성 향상, 재사용성 증가, 유지보수 용이성 증진 등의 이점을 얻을 수 있습니다.

SQLAlchemy ORM을 사용하지 않고, 표준 sqlite3 라이브러리를 사용하여 DB 의존성 코드를 구현한 예시

In [13]:
SessionLocal

sessionmaker(class_='Session', autocommit=False, bind=Engine(sqlite:///./mvc_app_item_setup.db), autoflush=False, expire_on_commit=True)

In [12]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

# ----------------------------------------------------
# 1. DB 설정 및 모델 정의 (Base 사용)
# ----------------------------------------------------

# SQLite 연결 설정 (mvc_app.db 파일을 사용)
SQLALCHEMY_DATABASE_URL = "sqlite:///./mvc_app_item_setup.db"

# 엔진 생성 (DB와의 소통 통로)
engine = create_engine(
    SQLALCHEMY_DATABASE_URL,
    # SQLite를 다중 스레드 환경에서 사용할 때 필요한 설정이지만,
    # 여기서는 단일 실행이므로 필수 아님. 습관적으로 포함.
    connect_args={"check_same_thread": False}
)

# 모든 ORM 모델의 기본 클래스
Base = declarative_base()

# Item 테이블 모델 정의
class Item(Base):
    __tablename__ = "items" # 실제 DB 테이블 이름

    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, index=True, unique=True)
    price = Column(Integer)
    description = Column(String, nullable=True)

# 세션 생성기 정의 (세션을 발급하는 공장)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# ----------------------------------------------------
# 2. 실행 로직: 테이블 생성 및 레코드 삽입
# ----------------------------------------------------

def setup_database():
    # 1. 테이블 생성
    # Base에 정의된 모든 모델(여기서는 Item)을 사용하여 DB에 테이블을 생성합니다.
    print("🚀 데이터베이스 파일 생성 및 테이블 구조 반영...")
    Base.metadata.create_all(bind=engine)

def insert_records():
    # 2. 세션 발급 및 트랜잭션 시작
    session = SessionLocal()
    print("⏳ 레코드 삽입 트랜잭션 시작...")

    try:
        # 삽입할 레코드 1
        item1 = Item(name="노트북", price=1200000, description="고성능 작업용 기기")
        # 삽입할 레코드 2
        item2 = Item(name="마우스", price=35000, description="무선 광마우스")

        # 세션에 객체 추가 (DB에 반영할 준비)
        session.add(item1)
        session.add(item2)

        # 트랜잭션 커밋 (DB에 실제로 저장)
        session.commit()
        print("✅ 레코드 2개 삽입 완료 (트랜잭션 커밋)")

    except Exception as e:
        # 오류 발생 시 롤백 (트랜잭션 취소)
        session.rollback()
        print(f"❌ 오류 발생. 롤백됨: {e}")

    finally:
        # 세션 종료 (리소스 반납)
        session.close()
        print("📚 세션 종료.")

# ----------------------------------------------------
# 3. 프로그램 실행
# ----------------------------------------------------
if __name__ == "__main__":
    setup_database()
    insert_records()

    # 확인을 위한 코드 (선택 사항)
    session = SessionLocal()
    count = session.query(Item).count()
    print(f"\n총 {count}개의 레코드가 DB에 저장되었습니다.")
    session.close()

🚀 데이터베이스 파일 생성 및 테이블 구조 반영...
⏳ 레코드 삽입 트랜잭션 시작...
✅ 레코드 2개 삽입 완료 (트랜잭션 커밋)
📚 세션 종료.

총 2개의 레코드가 DB에 저장되었습니다.


/tmp/ipython-input-1303078818.py:21: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


## **2. Session 객체**

SQLAlchemy의 Session 객체를 사용하여 데이터베이스 작업을 수행하는 기본적인 예제를 보여드리겠습니다. 이 예제는 일반적으로 FastAPI와 같은 웹 애플리케이션의 CRUD 함수에서 데이터베이스 로직을 구현할 때 사용되는 패턴입니다.

이 코드는 세션 설정, 모델 정의, 그리고 세션을 이용한 데이터 조회 및 생성의 세 부분으로 구성됩니다.

1. 설정 및 모델 정의

먼저, 데이터베이스 엔진을 설정하고, SessionLocal 클래스를 정의한 후, 사용할 ORM 모델(Item)을 정의합니다.

In [5]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import sessionmaker, Session
from sqlalchemy.ext.declarative import declarative_base

# ⚙️ 설정 (이전 질문에서 제공된 코드와 유사)
SQLALCHEMY_DATABASE_URL = "sqlite:///./example.db"
engine = create_engine(
    SQLALCHEMY_DATABASE_URL,
    connect_args={"check_same_thread": False}
)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base() # 모든 모델이 상속받을 기본 클래스

# 📦 모델 정의
class Item(Base):
    __tablename__ = "items"

    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, index=True)
    description = Column(String, nullable=True)

# 🚀 DB 초기화: 테이블 생성 (Base.metadata를 사용)
def init_db():
    Base.metadata.create_all(bind=engine)

init_db()

/tmp/ipython-input-1995538289.py:12: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base() # 모든 모델이 상속받을 기본 클래스


2. 세션(Session)을 이용한 CRUD 함수

여기서 db: Session 객체를 매개변수로 받아 실제 데이터베이스 작업을 수행합니다.

In [6]:
# Pydantic 모델을 사용한다고 가정하고 간단히 정의
class ItemCreate:
    name: str
    description: str | None = None

class ItemInDB:
    id: int
    name: str
    description: str | None = None


# 💾 데이터 생성 (CREATE)
def create_item(db: Session, item_data: ItemCreate) -> Item:
    # 1. ORM 모델 객체 생성
    db_item = Item(name=item_data.name, description=item_data.description)

    # 2. 세션에 추가 (데이터베이스에 INSERT할 준비)
    db.add(db_item)

    # 3. 트랜잭션 커밋 (실제 데이터베이스에 반영)
    db.commit()

    # 4. 객체 새로고침 (DB에서 생성된 ID를 객체에 반영)
    db.refresh(db_item)

    return db_item


# 🔎 데이터 조회 (READ)
def get_item_by_name(db: Session, item_name: str) -> Item | None:
    # 1. 세션의 query 객체를 사용하여 쿼리 작성 및 필터링
    # .first()는 조건에 맞는 첫 번째 결과만 반환합니다.
    item = db.query(Item).filter(Item.name == item_name).first()

    return item


# 📝 데이터 수정 및 삭제 (UPDATE, DELETE는 생략)
# 수정 예시:
# item_to_update = db.query(Item).filter(Item.id == item_id).first()
# if item_to_update:
#     item_to_update.description = "New Description"
#     db.commit()

3. 세션 관리 및 실행 (FastAPI의 get_db 패턴)

실제 세션 객체를 생성하고 위 함수들을 호출하며, **가장 중요한 세션 종료(close())**를 처리하는 부분입니다.

In [7]:
# 🔑 세션 관리 함수 (FastAPI의 Dependency Injection 패턴)
def get_session_context() -> Session:
    """
    세션을 열고, 사용 후 반드시 닫는 역할을 합니다.
    """
    db = SessionLocal() # 세션 생성
    try:
        # 여기서는 yield 대신 return을 사용하여 일반 함수처럼 동작하게 함
        return db
    finally:
        db.close() # 세션 종료 (연결 풀 반환)


# 🧪 실행 로직
print("--- DB 작업 시작 ---")

# 1. 세션 컨텍스트 얻기
db_session = get_session_context()

try:
    # ItemCreate 객체 대신 간단한 dict 사용 (실제로는 Pydantic 사용)
    new_item_data = ItemCreate()
    new_item_data.name = "Laptop"
    new_item_data.description = "A powerful computing device."

    # 2. 데이터 생성 함수 호출
    created_item = create_item(db=db_session, item_data=new_item_data)
    print(f"✅ 생성된 아이템 ID: {created_item.id}, 이름: {created_item.name}")

    # 3. 데이터 조회 함수 호출
    retrieved_item = get_item_by_name(db=db_session, item_name="Laptop")

    if retrieved_item:
        print(f"🔎 조회된 아이템: ID={retrieved_item.id}, Description='{retrieved_item.description}'")

except Exception as e:
    # 오류 발생 시 롤백 (트랜잭션 취소)
    db_session.rollback()
    print(f"❌ 오류 발생: {e}")

finally:
    # 4. 세션 닫기 (get_session_context의 finally 블록에서 처리됨)
    # 실제 이 코드는 FastAPI의 Depends(get_db) 메커니즘이 자동으로 처리합니다.
    db_session.close()

print("--- DB 작업 완료 ---")

--- DB 작업 시작 ---
✅ 생성된 아이템 ID: 1, 이름: Laptop
🔎 조회된 아이템: ID=1, Description='A powerful computing device.'
--- DB 작업 완료 ---


In [ ]:
from sklearn.linear_model import LogisticRegression